In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir('/content/drive/My Drive/')  # Укажите путь к папке, где находится ваш CSV-файл

In [ ]:
!pip install catboost -q
!pip install optuna -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.7 MB/s eta 0:00:00


In [ ]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# explosions/fires/damage

In [ ]:
import pandas as pd
from catboost import CatBoostClassifier, Pool, CatBoostRegressor


df = pd.read_csv("for_learning.csv")

# Список целевых меток, которые вы хотите удалить
target_columns = ['explosions/fires/damages'] #замените на свое

bad_features = ['accidents in transport', 'housing and utilities', 'accidents with hazardous/toxic substances emission',
                 'natural hazards', 'other hazards']

train_features = ['district','T', 'Po', 'P', 'Pa', 'U', 'Ff', 'ff10', 'ff3', 'N',
                  'Tn', 'Tx', 'Nh', 'VV', 'Td', 'RRR', 'tR', 'Tg', 'sss', 'Spring',
                  'Winter', 'Summer', 'Autumn', 'year', 'month', 'day', 'heating network',
                  'cold water network', 'hot water network', 'electricity network',
                  'gas network', 'water treatment stations', 'water pumping stations',
                  'water intake facilities', 'sewage network', 'sewage treatment plants',
                  'sewage pumping stations', 'boiler houses', 'heat points']


train = df.drop(bad_features, axis=1)

x = train.drop(target_columns, axis=1)
y = train.drop(train_features, axis=1)

class_counts = y.value_counts()

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from catboost import CatBoostRegressor, Pool, CatBoostClassifier

cat_features =['district']

train_pool = Pool(
    xtrain,
    ytrain,
    cat_features=cat_features

)
valid_pool = Pool(
    xtest,
    ytest,
    cat_features=cat_features
)

In [ ]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score

def objective(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 8),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        #od_wait=trial.suggest_int("od_wait", 10, 50),
        auto_class_weights =trial.suggest_categorical('auto_class_weights',['SqrtBalanced', 'Balanced']),
        verbose=False,

        eval_metric = 'TotalF1:use_weights=false;average=Macro',
        task_type="GPU",
        early_stopping_rounds = 100,
        use_best_model = True,
    )
    model.fit(train_pool, eval_set=valid_pool)
    y_pred = model.predict(xtest)
    return f1_score(ytest, y_pred, average='macro')

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
sampler = TPESampler(seed=1)

In [ ]:
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)

In [ ]:
study.optimize(objective, n_trials=50)

In [ ]:
print("Количество завершенных испытаний: ", len(study.trials))
print("Лучшее испытание:")
trial = study.best_trial
print("  Значение: ", trial.value)
print("  Параметры: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Количество завершенных испытаний:  50
Лучшее испытание:
  Значение:  0.49187464138888876
  Параметры: 
    iterations: 424
    learning_rate: 0.06076836762983255
    depth: 8
    l2_leaf_reg: 0.0004076241998149827
    bootstrap_type: Bayesian
    random_strength: 0.00026145629427797
    bagging_temperature: 9.502020499898281
    od_type: Iter
    auto_class_weights: SqrtBalanced


In [ ]:
catboost_params = {
    'learning_rate': 0.06076836762983255,
    'depth': 8,
    'l2_leaf_reg': 0.0004076241998149827,
    'bootstrap_type': 'Bayesian',
    'random_strength': 0.00026145629427797,
    'bagging_temperature': 9.502020499898281,
    'od_type': 'Iter',
    'task_type': 'GPU',
}

model = CatBoostRegressor(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.9916141	test: 0.9894737	best: 0.9894737 (0)	total: 82.8ms	remaining: 1m 22s
1:	learn: 0.9767028	test: 0.9749105	best: 0.9749105 (1)	total: 159ms	remaining: 1m 19s
2:	learn: 0.9743299	test: 0.9724626	best: 0.9724626 (2)	total: 215ms	remaining: 1m 11s
3:	learn: 0.9582773	test: 0.9556782	best: 0.9556782 (3)	total: 268ms	remaining: 1m 6s
4:	learn: 0.9567736	test: 0.9542921	best: 0.9542921 (4)	total: 329ms	remaining: 1m 5s
5:	learn: 0.9527865	test: 0.9502591	best: 0.9502591 (5)	total: 378ms	remaining: 1m 2s
6:	learn: 0.9523743	test: 0.9500014	best: 0.9500014 (6)	total: 417ms	remaining: 59.2s
7:	learn: 0.9520753	test: 0.9497837	best: 0.9497837 (7)	total: 462ms	remaining: 57.3s
8:	learn: 0.9476386	test: 0.9456346	best: 0.9456346 (8)	total: 503ms	remaining: 55.4s
9:	learn: 0.9433944	test: 0.9417285	best: 0.9417285 (9)	total: 548ms	remaining: 54.2s
10:	learn: 0.9318534	test: 0.9301947	best: 0.9301947 (10)	total: 587ms	remaining: 52.8s
11:	learn: 0.9316766	test: 0.9300114	best: 0.930

In [ ]:
import pickle

pickle.dump(model, open('explosions_fires_damages.pkl', 'wb'))

# natural hazards

In [ ]:
import pandas as pd
df = pd.read_csv("for_learning.csv")

# Список целевых меток, которые вы хотите удалить
target_columns = ['natural hazards'] #замените на свое

bad_features = ['accidents in transport', 'housing and utilities', 'accidents with hazardous/toxic substances emission',
                 'explosions/fires/damages', 'other hazards']

train_features = ['district','T', 'Po', 'P', 'Pa', 'U', 'Ff', 'ff10', 'ff3', 'N',
                  'Tn', 'Tx', 'Nh', 'VV', 'Td', 'RRR', 'tR', 'Tg', 'sss', 'Spring',
                  'Winter', 'Summer', 'Autumn', 'year', 'month', 'day', 'heating network',
                  'cold water network', 'hot water network', 'electricity network',
                  'gas network', 'water treatment stations', 'water pumping stations',
                  'water intake facilities', 'sewage network', 'sewage treatment plants',
                  'sewage pumping stations', 'boiler houses', 'heat points']


train = df.drop(bad_features, axis=1)

x = train.drop(target_columns, axis=1)
y = train.drop(train_features, axis=1)

class_counts = y.value_counts()

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score

def objective(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 8),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        #od_wait=trial.suggest_int("od_wait", 10, 50),
        auto_class_weights =trial.suggest_categorical('auto_class_weights',['SqrtBalanced', 'Balanced']),
        verbose=False,

        eval_metric = 'TotalF1:use_weights=false;average=Macro',
        task_type="GPU",
        early_stopping_rounds = 100,
        use_best_model = True,
    )
    model.fit(train_pool, eval_set=valid_pool)
    y_pred = model.predict(xtest)
    return f1_score(ytest, y_pred, average='macro')

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
sampler = TPESampler(seed=1)

In [ ]:
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)

In [ ]:
study.optimize(objective, n_trials=50, show_progress_bar=True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
print("Количество завершенных испытаний: ", len(study.trials))
print("Лучшее испытание:")
trial = study.best_trial
print("  Значение: ", trial.value)
print("  Параметры: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Количество завершенных испытаний:  50
Лучшее испытание:
  Значение:  0.20259454753273803
  Параметры: 
    iterations: 115
    learning_rate: 0.0023479783654800504
    depth: 4
    l2_leaf_reg: 3.3871557440085103
    bootstrap_type: Bayesian
    random_strength: 1.5778974699502737
    bagging_temperature: 2.5932225489517595
    od_type: Iter
    auto_class_weights: SqrtBalanced


In [ ]:
catboost_params = {
    'learning_rate': 0.0023479783654800504,
    'depth': 4,
    'l2_leaf_reg': 3.3871557440085103,
    'bootstrap_type': 'Bayesian',
    'random_strength': 1.5778974699502737,
    'bagging_temperature': 2.5932225489517595,
    'od_type': 'Iter',
    'task_type': 'GPU',
}

model = CatBoostRegressor(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 1.0112149	test: 1.0098684	best: 1.0098684 (0)	total: 19.6ms	remaining: 19.5s
1:	learn: 1.0104100	test: 1.0090594	best: 1.0090594 (1)	total: 37.2ms	remaining: 18.6s
2:	learn: 1.0096316	test: 1.0082728	best: 1.0082728 (2)	total: 63.7ms	remaining: 21.2s
3:	learn: 1.0088503	test: 1.0074854	best: 1.0074854 (3)	total: 84.1ms	remaining: 20.9s
4:	learn: 1.0080979	test: 1.0066965	best: 1.0066965 (4)	total: 117ms	remaining: 23.3s
5:	learn: 1.0073164	test: 1.0059089	best: 1.0059089 (5)	total: 175ms	remaining: 29s
6:	learn: 1.0065494	test: 1.0051430	best: 1.0051430 (6)	total: 212ms	remaining: 30.1s
7:	learn: 1.0058263	test: 1.0044122	best: 1.0044122 (7)	total: 254ms	remaining: 31.4s
8:	learn: 1.0050825	test: 1.0036681	best: 1.0036681 (8)	total: 292ms	remaining: 32.1s
9:	learn: 1.0043013	test: 1.0028833	best: 1.0028833 (9)	total: 325ms	remaining: 32.2s
10:	learn: 1.0035203	test: 1.0021046	best: 1.0021046 (10)	total: 362ms	remaining: 32.5s
11:	learn: 1.0027750	test: 1.0013616	best: 1.00136

In [ ]:
import pickle

pickle.dump(model, open('natural_hazards.pkl', 'wb'))

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
from catboost import CatBoostClassifier, Pool

n_splits = 3
clfs = []
scores = []

catboost_params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'depth': 6,
    'l2_leaf_reg': 3,
    'random_state': 42,
    'eval_metric': 'TotalF1:average=Macro',
    'verbose': False
}

kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=7575)
for train_index, test_index in kf.split(x,y):

    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    cat_features =['district']

    train_dataset = Pool(
        X_train,
        y_train,
        cat_features=cat_features
    )
    eval_dataset = Pool(
        X_test,
        y_test,
        cat_features=cat_features
    )

    clf = CatBoostClassifier(**catboost_params)

    clfs.append(clf)

    clf.fit(
        train_dataset,
        eval_set=eval_dataset,
        verbose=500,
        use_best_model=True,
        plot=False)

    scores.append(np.mean([v for k, v in clf.best_score_["validation"].items() if "TotalF1:average=Macro" in k], dtype="float16"))

assert len(clfs) == n_splits
print("mean TotalF1:use_weights=false;average=Macro score --------->", np.mean(scores, dtype="float16") - np.std(scores, dtype="float16"))

0:	learn: 0.3116532	test: 0.3082395	best: 0.3082395 (0)	total: 76ms	remaining: 1m 15s
500:	learn: 0.9511187	test: 0.9097749	best: 0.9100925 (497)	total: 39.1s	remaining: 39s
999:	learn: 0.9866524	test: 0.9342029	best: 0.9343861 (991)	total: 1m 17s	remaining: 0us

bestTest = 0.9343860736
bestIteration = 991

Shrink model to first 992 iterations.
0:	learn: 0.2734888	test: 0.2714356	best: 0.2714356 (0)	total: 130ms	remaining: 2m 9s
500:	learn: 0.9561182	test: 0.8968026	best: 0.8970132 (498)	total: 36.8s	remaining: 36.6s
999:	learn: 0.9861034	test: 0.9182476	best: 0.9183189 (998)	total: 1m 17s	remaining: 0us

bestTest = 0.9183189166
bestIteration = 998

Shrink model to first 999 iterations.
0:	learn: 0.2698968	test: 0.2682834	best: 0.2682834 (0)	total: 69.7ms	remaining: 1m 9s
500:	learn: 0.9617005	test: 0.9126110	best: 0.9126110 (500)	total: 44.6s	remaining: 44.4s
999:	learn: 0.9883455	test: 0.9260510	best: 0.9260510 (999)	total: 1m 26s	remaining: 0us

bestTest = 0.9260510393
bestIteration

# Accidents  in transport

In [ ]:
import pandas as pd
df = pd.read_csv("for_learning.csv")

# Список целевых меток, которые вы хотите удалить
target_columns = ['accidents in transport'] #замените на свое

bad_features = ['natural hazards', 'housing and utilities', 'accidents with hazardous/toxic substances emission',
                 'explosions/fires/damages', 'other hazards']

train_features = ['district','T', 'Po', 'P', 'Pa', 'U', 'Ff', 'ff10', 'ff3', 'N',
                  'Tn', 'Tx', 'Nh', 'VV', 'Td', 'RRR', 'tR', 'Tg', 'sss', 'Spring',
                  'Winter', 'Summer', 'Autumn', 'year', 'month', 'day', 'heating network',
                  'cold water network', 'hot water network', 'electricity network',
                  'gas network', 'water treatment stations', 'water pumping stations',
                  'water intake facilities', 'sewage network', 'sewage treatment plants',
                  'sewage pumping stations', 'boiler houses', 'heat points']


train = df.drop(bad_features, axis=1)

x = train.drop(target_columns, axis=1)
y = train.drop(train_features, axis=1)

class_counts = y.value_counts()

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score

def objective(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 8),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        #od_wait=trial.suggest_int("od_wait", 10, 50),
        auto_class_weights =trial.suggest_categorical('auto_class_weights',['SqrtBalanced', 'Balanced']),
        verbose=False,

        eval_metric = 'TotalF1:use_weights=false;average=Macro',
        task_type="GPU",
        early_stopping_rounds = 100,
        use_best_model = True,
    )
    model.fit(train_pool, eval_set=valid_pool)
    y_pred = model.predict(xtest)
    return f1_score(ytest, y_pred, average='macro')

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
sampler = TPESampler(seed=1)

In [ ]:
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)

In [ ]:
study.optimize(objective, n_trials=50, show_progress_bar=True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
print("Количество завершенных испытаний: ", len(study.trials))
print("Лучшее испытание:")
trial = study.best_trial
print("  Значение: ", trial.value)
print("  Параметры: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Количество завершенных испытаний:  50
Лучшее испытание:
  Значение:  0.23067708068875317
  Параметры: 
    iterations: 370
    learning_rate: 0.0016470796257226222
    depth: 5
    l2_leaf_reg: 0.7624522335720814
    bootstrap_type: Bayesian
    random_strength: 0.0009437959809039722
    bagging_temperature: 1.1520392831847683
    od_type: IncToDec
    auto_class_weights: SqrtBalanced


In [ ]:
catboost_params = {
    'learning_rate': 0.0016470796257226222,
    'depth': 5,
    'l2_leaf_reg': 0.7624522335720814,
    'bootstrap_type': 'Bayesian',
    'random_strength': 0.0009437959809039722,
    'bagging_temperature': 1.1520392831847683,
    'od_type': 'IncToDec',
    'task_type': 'GPU',
}

model = CatBoostRegressor(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 1.0113429	test: 1.0100427	best: 1.0100427 (0)	total: 30ms	remaining: 29.9s
1:	learn: 1.0107504	test: 1.0094515	best: 1.0094515 (1)	total: 54.7ms	remaining: 27.3s
2:	learn: 1.0101550	test: 1.0088561	best: 1.0088561 (2)	total: 79ms	remaining: 26.3s
3:	learn: 1.0095535	test: 1.0082616	best: 1.0082616 (3)	total: 103ms	remaining: 25.6s
4:	learn: 1.0089607	test: 1.0076737	best: 1.0076737 (4)	total: 127ms	remaining: 25.2s
5:	learn: 1.0083623	test: 1.0070825	best: 1.0070825 (5)	total: 150ms	remaining: 24.8s
6:	learn: 1.0077656	test: 1.0064929	best: 1.0064929 (6)	total: 175ms	remaining: 24.8s
7:	learn: 1.0071704	test: 1.0059049	best: 1.0059049 (7)	total: 198ms	remaining: 24.5s
8:	learn: 1.0065849	test: 1.0053229	best: 1.0053229 (8)	total: 221ms	remaining: 24.3s
9:	learn: 1.0059941	test: 1.0047350	best: 1.0047350 (9)	total: 247ms	remaining: 24.5s
10:	learn: 1.0054107	test: 1.0041564	best: 1.0041564 (10)	total: 276ms	remaining: 24.8s
11:	learn: 1.0048338	test: 1.0035773	best: 1.0035773 

In [ ]:
import pickle

pickle.dump(model, open('accidents_in_transport.pkl', 'wb'))

# Accidents with hazardous/toxic substances emission

In [ ]:
import pandas as pd
df = pd.read_csv("for_learning.csv")

# Список целевых меток, которые вы хотите удалить
target_columns = ['accidents with hazardous/toxic substances emission'] #замените на свое

bad_features = ['natural hazards', 'housing and utilities', 'accidents in transport',
                 'explosions/fires/damages', 'other hazards']

train_features = ['district','T', 'Po', 'P', 'Pa', 'U', 'Ff', 'ff10', 'ff3', 'N',
                  'Tn', 'Tx', 'Nh', 'VV', 'Td', 'RRR', 'tR', 'Tg', 'sss', 'Spring',
                  'Winter', 'Summer', 'Autumn', 'year', 'month', 'day', 'heating network',
                  'cold water network', 'hot water network', 'electricity network',
                  'gas network', 'water treatment stations', 'water pumping stations',
                  'water intake facilities', 'sewage network', 'sewage treatment plants',
                  'sewage pumping stations', 'boiler houses', 'heat points']


train = df.drop(bad_features, axis=1)

x = train.drop(target_columns, axis=1)
y = train.drop(train_features, axis=1)

class_counts = y.value_counts()

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

FileNotFoundError: ignored

In [ ]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score

def objective(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 8),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        #od_wait=trial.suggest_int("od_wait", 10, 50),
        auto_class_weights =trial.suggest_categorical('auto_class_weights',['SqrtBalanced', 'Balanced']),
        verbose=False,

        eval_metric = 'TotalF1:use_weights=false;average=Macro',
        task_type="GPU",
        early_stopping_rounds = 100,
        use_best_model = True,
    )
    model.fit(train_pool, eval_set=valid_pool)
    y_pred = model.predict(xtest)
    return f1_score(ytest, y_pred, average='macro')

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
sampler = TPESampler(seed=1)

In [ ]:
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)

In [ ]:
study.optimize(objective, n_trials=50, show_progress_bar=True)

In [ ]:
print("Количество завершенных испытаний: ", len(study.trials))
print("Лучшее испытание:")
trial = study.best_trial
print("  Значение: ", trial.value)
print("  Параметры: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
catboost_params = {
    'learning_rate': 0.0016470796257226222,
    'depth': 5,
    'l2_leaf_reg': 0.7624522335720814,
    'bootstrap_type': 'Bayesian',
    'random_strength': 0.0009437959809039722,
    'bagging_temperature': 1.1520392831847683,
    'od_type': 'IncToDec',
    'task_type': 'GPU',
}

model = CatBoostRegressor(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

In [ ]:
import pickle

pickle.dump(model, open('accidents_with_hazardous_toxic substances emission.pkl', 'wb'))